In [1]:
from sklearn import metrics, cluster

import pandas as pd
import numpy as np

Представим, что международное круизное агентство Carnival Cruise Line решило себя разрекламировать с помощью баннеров и обратилось для этого к вам. Чтобы протестировать, велика ли от таких баннеров польза, их будет размещено всего 20 штук по всему миру. Вам надо выбрать 20 таких локаций для размещения, чтобы польза была большой и агентство продолжило с вами сотрудничать.

Агентство крупное, и у него есть несколько офисов по всему миру. Вблизи этих офисов оно и хочет разместить баннеры — легче договариваться и проверять результат. Также эти места должны быть популярны среди туристов.

Для поиска оптимальных мест воспользуемся базой данных крупнейшей социальной сети, основанной на локациях — Foursquare.

Часть открытых данных есть, например, на сайте archive.org:

https://archive.org/details/201309_foursquare_dataset_umn

Скачаем любым удобным образом архив fsp.zip с этой страницы.

Нас будет интересовать файл checkins.dat. Для удобной работы с этим документом преобразуем его к формату csv, удалив строки, не содержащие координат — они неинформативны для нас:

In [16]:
with open("checkins.dat", "r") as dat:
    cnt = 1
    for line in dat:
        if cnt == 1:
            header = line.split('|')
            header = [i.strip() for i in header]
            Frame = []
            Frame.append(header)
        else:
            if cnt != 2:
                flag = True
                tmp = line.split('|')
                array = [i.strip() for i in tmp]
                for i in array:
                    if len(i) == 0:
                        flag = False
                if flag == True:
                    Frame.append(array)
        cnt += 1

In [22]:
data = pd.DataFrame(Frame[1:][:], columns=Frame[:][0])

Теперь необходимо кластеризовать данные координаты, чтобы выявить центры скоплений туристов. Поскольку баннеры имеют сравнительно небольшую площадь действия, нам нужен алгоритм, позволяющий ограничить размер кластера и не зависящий от количества кластеров.

Эта задача — хороший повод познакомиться с алгоритмом MeanShift, который мы обошли стороной в основной части лекций. Его описание при желании можно посмотреть в sklearn user guide, а чуть позже появится дополнительное видео с обзором этого и некоторых других алгоритмов кластеризации. Используйте MeanShift, указав bandwidth=0.1, что в переводе из градусов в метры колеблется примерно от 5 до 10 км в средних широтах.

Примечание: на 396634 строках кластеризация будет работать долго. Быть очень терпеливым не возбраняется — результат от этого только улучшится. Но для того, чтобы сдать задание, понадобится сабсет из первых 100 тысяч строк. Это компромисс между качеством и затраченным временем. Обучение алгоритма на всём датасете занимает около часа, а на 100 тыс. строк — примерно 2 минуты, однако этого достаточно для получения корректных результатов.

In [48]:
y = []
X = data[:][:100000]
X = X[['latitude', 'longitude']]
algo = cluster.MeanShift(bandwidth=0.1)
algo.fit_predict(X)
y = algo.predict(X)

Некоторые из получившихся кластеров содержат слишком мало точек — такие кластеры не интересны рекламодателям. Поэтому надо определить, какие из кластеров содержат, скажем, больше 15 элементов. Центры этих кластеров и являются оптимальными для размещения.

In [49]:
dict = {}
for i in y:
    if i in dict:
        dict[i] += 1
    else:
        dict[i] = 0

In [55]:
clusters = []
for i in dict:
    if dict[i] > 15:
        clusters.append(i)
centers = algo.cluster_centers_[clusters]

In [56]:
centers

array([[  40.7177164 ,  -73.99183542],
       [  33.44943805, -112.00213969],
       [  33.44638027, -111.90188756],
       ..., 
       [  41.61853175,  -88.44556818],
       [  38.65877915,  -76.8856871 ],
       [  39.2494686 ,  -77.1821271 ]])

Как мы помним, 20 баннеров надо разместить близ офисов компании. Найдем на Google Maps по запросу Carnival Cruise Line адреса офисов:

In [59]:
office = [
    [33.751277, -118.188740],
    [25.867736, -80.324116],
    [51.503016, -0.075479],
    [52.378894, 4.885084],
    [39.366487, 117.036146],
    [-33.868457, 151.205134],
]

Осталось определить 20 ближайших к ним центров кластеров. Т.е. посчитать дистанцию до ближайшего офиса для каждой точки и выбрать 20 с наименьшим значением.

Примечание: при подсчете расстояний и в кластеризации можно пренебречь тем, что Земля круглая, так как в точках, расположенных близко друг к другу погрешность мала, а в остальных точках значение достаточно велико.

Для сдачи задания выберите из получившихся 20 центров тот, который наименее удален от ближайшего к нему офиса. Ответ в этом задании — широта и долгота этого центра, записанные через пробел.

In [62]:
def metrics(a, b):
    return ((a[0] - b[0])**2 + (a[1] - b[1])**2)**0.5

In [68]:
array = []
for i in range(centers.shape[0]):
    min = metrics(centers[i], office[0])
    for j in office:
        if metrics(centers[i],j) < min:
            min = metrics(centers[i],j)
    array.append([min, i])

In [69]:
array.sort()

[[0.007834758163107856, 408],
 [0.0093533161859922255, 373],
 [0.022674066158385495, 405],
 [0.050058294822787869, 58],
 [0.070847732427199731, 51],
 [0.13410903336184654, 29],
 [0.1674059642503429, 166],
 [0.18887596060185083, 92],
 [0.19577945647763628, 87],
 [0.21181053682436798, 42],
 [0.22223329073179071, 284],
 [0.27130075950667348, 314],
 [0.29497888680045692, 119],
 [0.30227011869246051, 55],
 [0.30473050307840693, 27],
 [0.31488379033627317, 11],
 [0.33881047025113181, 32],
 [0.34084565332205718, 158],
 [0.37868750125029754, 17],
 [0.38670622484272771, 47],
 [0.39877802665198009, 64],
 [0.40998299415480599, 24],
 [0.41041794393202896, 35],
 [0.4480553156552331, 50],
 [0.49338744477858126, 100],
 [0.54047052278595753, 144],
 [0.57420460830214937, 90],
 [0.60309252588281059, 75],
 [0.61907009110124445, 223],
 [0.63853414350702598, 283],
 [0.67104887104407729, 285],
 [0.6711589396151707, 392],
 [0.73291837687442563, 143],
 [0.75971416617112086, 231],
 [0.78730149371138214, 492],


In [72]:
with open("placement_of_bann.txt", "w") as fout:
    fout.write(str(centers[array[0][1]]))